In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder

/var/folders/tl/tcgv4w050yj5nx3f9m3gwx6w0000gn/T/ipykernel_19859/3242553389.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
ucs_df = pd.read_csv('used_car_sales.csv')
print(ucs_df.head(30).to_markdown())

|    |     ID |   pricesold |   yearsold | zipcode   |   Mileage | Make              | Model              |   Year | Trim                                        | Engine                          | BodyType             |   NumCylinders | DriveType   |
|---:|-------:|------------:|-----------:|:----------|----------:|:------------------|:-------------------|-------:|:--------------------------------------------|:--------------------------------|:---------------------|---------------:|:------------|
|  0 | 137178 |        7500 |       2020 | 786**     |     84430 | Ford              | Mustang            |   1988 | LX                                          | 5.0L Gas V8                     | Sedan                |              0 | RWD         |
|  1 |  96705 |       15000 |       2019 | 81006     |         0 | Replica/Kit Makes | Jaguar Beck Lister |   1958 | nan                                         | 383 Fuel injected               | Convertible          |              8 | RWD       

In [ ]:
pd.options.display.max_rows = 4000

In [ ]:
ucs_df['BodyType'].value_counts()

In [ ]:
ucs_df = ucs_df[["pricesold", "yearsold", "Mileage", "Make", "Year", "BodyType"]]

In [ ]:
ucs_df

In [ ]:
ucs_df['Mileage'].hist()

In [ ]:
ucs_df['Mileage'].max()

In [ ]:
# remove cars with more than 400k miles
ucs_df = ucs_df[ucs_df['Mileage'] < 400000]

In [ ]:
ucs_df['Mileage'].hist()

In [ ]:
ucs_df['Year'].max()

In [ ]:
ucs_df = ucs_df[(ucs_df['Year'] < 2024) & (ucs_df['Year'] > 1910)]

In [ ]:
ucs_df['Year'].hist()

In [ ]:
unique_makes = list(ucs_df['Make'].unique())

print(len(unique_makes))
print(*sorted(unique_makes), sep = "\n")


In [ ]:
list(ucs_df['Make'].value_counts().index[0:80])

In [ ]:
all_makes = list(ucs_df['Make'].unique())

for make in all_makes:
    if 'chry' in make.lower():
        print(make)

In [ ]:
good_makes = list(ucs_df['Make'].value_counts().index[0:80])

make_cleanup_map = {'austin': 'Austin',
 'bmw': 'BMW',
 'chev': 'Chevrolet',
 'chry': 'Chrysler',
 'diamond': 'Diamond T',
 'dodge': 'Ram',
 'fiat': 'Fiat',
 'ford': 'Ford',
 'gmc': 'GMC',
 'honda': 'Honda',
 'hummer': 'Hummer',
 'jaguar': 'Jaguar',
 'jeep': 'Jeep',
 'lexus': 'Lexus',
 'lincoln': 'Lincoln',
 'lotus': 'Lotus',
 'mazda': 'Mazda',
 'mercedes': 'Mercedes-Benz',
 'mouth': 'Chrysler',
 'nissan': 'Nissan',
 'olds': 'Oldsmobile',
 'pon': 'Pontiac',
 'ram': 'Ram',
 'rambler': 'AMC',
 'rover': 'Land Rover',
 'saturn': 'Saturn',
 'subaru': 'Subaru',
 'suburu': 'Subaru',
 'tesla': 'Tesla',
 'toyota': 'Toyota',
 'volks': 'Volkswagen',
 'volvo': 'Volvo',
 'vw': 'Volkswagen'
 }

def make_cleanup(orig_make):
    make_lower = orig_make.lower()

    if orig_make in good_makes:
        return orig_make
    else:
        for key in make_cleanup_map.keys():
            if key in make_lower:
                return make_cleanup_map[key]

    return 'Other Makes'

ucs_df['clean_make'] = ucs_df['Make'].apply(lambda x: make_cleanup(x))


In [ ]:
ucs_df['clean_make'].value_counts()

In [ ]:
ucs_df['BodyType'] = ucs_df['BodyType'].astype(str)

In [ ]:
np.random.seed(42)

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(ucs_df['BodyType'])

kmeans = KMeans(n_clusters=8, random_state=42)

kmeans.fit(X)

body_df = ucs_df[['BodyType']].copy()
body_df['cluster'] = kmeans.labels_

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    for i in range(8):
        print(body_df[body_df['cluster'] == i].head(500))
        

In [ ]:
body_cleanup_map = {
    'van': 'Van',
    'pickup': 'Pickup', 
    'cab': 'Pickup',
    'bed': 'Pickup',
    'truck': 'Pickup',
    'coupe': 'Coupe',
    '2door': 'Coupe',
    '2dr': 'Coupe',
    'sedan': 'Sedan',
    '4door': 'Sedan',
    '4dr': 'Sedan',
    'suv' : 'SUV',
    'jeep': 'SUV',
    'sportutil': 'SUV',
    'convertible': 'Convertible',
    'wagon': 'Wagon',
    'hatchback': 'Hatchback',
    'fastback': 'Fastback'
}

def body_cleanup(orig_body):
    body_stripped = ''.join(e for e in orig_body if e.isalnum()).lower()

    for key in body_cleanup_map.keys():
        if key in body_stripped:
            return body_cleanup_map[key]
    return 'Other Bodies'

ucs_df['clean_body'] = ucs_df['BodyType'].apply(lambda x: body_cleanup(x))

In [ ]:

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(ucs_df[['BodyType', 'clean_body']][ucs_df['clean_body'] == 'Other Bodies'])

In [ ]:
counts = ucs_df['clean_body'].value_counts()

counts.plot(kind='bar', color='blue')

plt.grid(False)

plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)

plt.xticks(rotation=90)

plt.show()

In [ ]:
modes = ucs_df.mode().iloc[0]

def proc_data(df):
    df['CleanMake'] = pd.Categorical(df.clean_make)
    df['CleanBody'] = pd.Categorical(df.clean_body)
    df['YearMade'] = pd.Categorical(df.Year)
    df['YearSold'] = pd.Categorical(df.yearsold)
    
    df.fillna(modes, inplace=True)

proc_data(ucs_df)

In [ ]:
cons_df = ucs_df[['pricesold', 'YearSold', 'Mileage', 'YearMade', 'CleanMake', 'CleanBody']]

In [ ]:
categorical_cols = ['YearSold', 'YearMade', 'CleanMake', 'CleanBody']
le = LabelEncoder() 
cons_df[categorical_cols] = cons_df[categorical_cols].apply(lambda col: le.fit_transform(col))

In [ ]:
cons_df

In [ ]:
X = cons_df.drop(columns=['pricesold'])
pricesold_column = cons_df.loc[:, 'pricesold']
y = pricesold_column.values


In [ ]:
rf = RandomForestRegressor(n_estimators=300)
rf.fit(X, y)

In [ ]:
rf.feature_importances_

In [ ]:
plt.barh(X.columns, rf.feature_importances_)